In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow
import zipfile as zf
import os
import csv
import gc
import operator
import random
from sklearn.cross_validation import train_test_split
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from random import shuffle
from IPython.display import Image
from pathlib import Path
import tensorflow as tf

C:\Users\PISME_Public01\Anaconda3\envs\tf\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.utils import np_utils
import keras.backend as K
from keras.models import Sequential
from keras import optimizers

Using TensorFlow backend.


In [3]:
from tqdm import tqdm, trange

In [4]:
# file name
TRAIN_FILE = '../data/train/'
TRAIN_CSV = '../data/train.csv'

In [5]:
train_csv = pd.read_csv(TRAIN_CSV)
train_csv.sample(10)

,Image,Id
9569,f87447c6.jpg,w_bbf6ebf
6831,b1519624.jpg,w_569c2c7
7699,c7c249d8.jpg,w_309a2b3
5247,88532e70.jpg,w_b7a832a
1837,2ff1d55b.jpg,w_1fd0d0e
3633,5efae38d.jpg,w_f2e9055
5054,83925a8c.jpg,w_f8cf8ca
4032,69dd7149.jpg,w_17964ef
8737,e38a5d8f.jpg,w_2707bcd
571,0e2bee83.jpg,w_77e1d3f


In [6]:
def prepareImages(data, m, dataset):
    
    print("Preparing images")
    
    X_train = np.zeros((m, 128, 128, 3))
    
    count = 0
    for fig in tqdm(data['Image']):
        img = image.load_img(f'{TRAIN_FILE}/{dataset}/{fig}', target_size=(128, 128, 3)) # keras.preprocessing.image.load_img 导入图像并且resize
        x = image.img_to_array(img) # keras.preprocessing.image.img_to_array img转化成arr
        
        x = preprocess_input(x) # keras.applications.imagenet_utils.preprocess_input对图像进行归一化
        X_train[count] = x
        
        count += 1
    
    count = 0
    print("Finished")
    
    return X_train

In [21]:
def prepareY(Y):
    
    values = array(Y)
    print(values.shape)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    print(integer_encoded)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print(onehot_encoded)
    
    y = onehot_encoded
    print(y.shape)
    return y, label_encoder

In [8]:
mod = Sequential()

mod.add(Conv2D(32, (7, 7), strides=(1, 1), name='conv0', input_shape=(128, 128, 3)))

mod.add(BatchNormalization(axis=3, name='bn0'))
mod.add(Activation('relu'))

mod.add(MaxPooling2D((2, 2), name='max_pool'))
mod.add(Conv2D(64, (3, 3), strides=(1,1), name='conv1'))
mod.add(Activation('relu'))
mod.add(AveragePooling2D((3, 3), name='avg_pool'))

mod.add(Flatten())
mod.add(Dense(500, activation='relu', name='rl'))
mod.add(Dropout(0.8))
mod.add(Dense(4251, activation='softmax', name='sm'))
print(mod.output_shape)

#opt = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
mod.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

(None, 4251)


In [9]:
?Dense

In [10]:
X = prepareImages(train_csv, 9850, "train")

Preparing images


100%|██████████████████████████████████████| 9850/9850 [05:09<00:00, 31.81it/s]


Finished


In [17]:
X /= 255
print(f"Shape of X-train: {X.shape}")

Shape of X-train: (9850, 128, 128, 3)


In [22]:
Y = train_csv['Id']
print(f'Shape of Y-train: {Y.shape}')

y, label_encoder = prepareY(Y)

Shape of Y-train: (9850,)
(9850,)
[3741  310 3613 ..., 2337  746 3088]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(9850, 4251)


In [24]:
history = mod.fit(X, y, epochs=100, batch_size=100, verbose=1)

Epoch 1/100
1000/9850 [==>...........................] - ETA: 30:14 - loss: 8.3543 - acc: 0.0000e+ - ETA: 30:34 - loss: 8.3539 - acc: 0.0000e+ - ETA: 28:30 - loss: 8.3524 - acc: 0.0000e+ - ETA: 25:56 - loss: 8.3418 - acc: 0.0125   - ETA: 24:24 - loss: 8.3364 - acc: 0.02 - ETA: 23:32 - loss: 8.3100 - acc: 0.04 - ETA: 22:31 - loss: 8.3117 - acc: 0.05 - ETA: 21:42 - loss: 8.2874 - acc: 0.05 - ETA: 21:03 - loss: 8.2988 - acc: 0.05 - ETA: 20:32 - loss: 8.2781 - acc: 0.0590

KeyboardInterrupt: 

In [23]:
?mod.fit